# SET the Image Resolution

In [1]:
#from helper import IMAGE_RES
IMAGE_RES = 100 #75

In [2]:
import cv2
import time
import json
import keras
import tensorflow
import numpy as np
from PIL import Image
import mediapipe as mp

In [3]:
file_name = "data_lookup_train.json"

# Load the JSON data from the file into a dictionary
with open(file_name, "r") as json_file:
    lookup = json.load(json_file)

lookup = {str(value): key.upper() for key, value in lookup.items()}
print(lookup)

{'0': 'ANGRY', '1': 'DISGUST', '2': 'FEAR', '3': 'HAPPY', '4': 'NEUTRAL', '5': 'SAD', '6': 'SURPRISE'}


In [4]:
from tensorflow.keras import layers, models


num_classes = len(lookup.keys())



model = models.Sequential(
    [
        layers.Conv2D(
            32, (3, 3), activation="relu", input_shape=(IMAGE_RES, IMAGE_RES, 3)
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()


model.load_weights("model_weights.h5")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 28224)             0

In [5]:
# Initialize MediaPipe Face Mesh solution
mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
    max_num_faces=1,  # Detect only one face
    refine_landmarks=True,  # Improve landmark accuracy
    min_detection_confidence=0.5,
)

# Initialize webcam
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)

    # Convert image to RGB format (MediaPipe expects RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process image with MediaPipe Face Mesh
    results = mp_face_mesh.process(frame_rgb)

    # Draw boundary boxes for detected faces
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Get the bounding box coordinates for the face
            x_min, y_min, x_max, y_max = 10000, 10000, -1, -1
            for landmark in face_landmarks.landmark:
                x, y = int(landmark.x * frame.shape[1]), int(
                    landmark.y * frame.shape[0]
                )
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x)
                y_max = max(y_max, y)

            # Draw the bounding box on the image
            cv2.rectangle(
                frame,
                (x_min - 50, y_min - 50),
                (x_max + 50, y_max + 50),
                (0, 255, 0),
                2,
            )
            # Draw the label
            label = cv2.rectangle(
                frame,
                (x_min - 50, y_min - 50),
                (x_max + 50, y_max + 50),
                (0, 255, 0),
                2,
            )
            start = time.time()
            # Crop the region of interest
            img_crop = frame[y_min - 50 : y_max + 50, x_min - 50 : x_max + 50]
            try:
                img_crop = Image.fromarray(np.uint8(img_crop))
                img_crop = img_crop.resize((IMAGE_RES, IMAGE_RES))
                img_crop = np.array(img_crop)
                img_crop = np.fliplr(img_crop)
                img_crop = np.array(
                    img_crop[:, :, ::-1], dtype="float32"
                )  # convert bgr to rgb
                img_crop = img_crop / 255
                # plt.imshow(img_crop)
                # plt.show()
                img_crop = img_crop.reshape((1, IMAGE_RES, IMAGE_RES, 3))

                # test_datagen = ImageDataGenerator()
                # test_generator = test_datagen.flow(img_crop, shuffle=False)

                predict_test = model.predict(img_crop)
                # print("Raw Predictions:", predict_test)

                predicted_label = np.argmax(predict_test)
                #print(predicted_label)

                end = time.time()
                cv2.putText(
                    label,
                    # f'Label: {str(directory_reverse_lookup[str(predicted_label)])}',
                    f"Label: {lookup[str(predicted_label)]}",
                    (20, 60),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.9,
                    (36, 255, 12),
                    2,
                )
                # label= cv2.flip(label,1)
                # print(predicted_label)

                fps = 1 / (end - start)

                # string formatting for fps
                cv2.putText(
                    frame,
                    f"FPS: {fps:.2f}",
                    (20, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.9,
                    (36, 255, 12),
                    2,
                )
            except Exception as E:
                print("Outside", E)

            # Display the cropped region
            # cv2.imshow("Cropped Region", roi)

    # Display the image with bounding boxes
    cv2.imshow("Face Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 20ms/step
